# Many thanks to these amazing notebooks created by:

@ambrosm 
- the nice EDA for this dataset conveyed per https://www.kaggle.com/ambrosm/tpsjan22-01-eda-which-makes-sense
- the feature engineering routines invented per https://www.kaggle.com/ambrosm/tpsjan22-03-linear-model

@gvyshnya
-- uses featurewiz to select 200 features out of ~625 features https://www.kaggle.com/gvyshnya/jan22-tpc-feature-importance-with-featurewiz/comments

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import pickle
import itertools
import gc
import math
from typing import Tuple, List, Dict
import matplotlib.dates as md
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
import dateutil.easter as easter


In [ ]:
train_path = '../input/tabular-playground-series-jan-2022/train.csv'
test_path = '../input/tabular-playground-series-jan-2022/test.csv'
sample_submission_path = '../input/tabular-playground-series-jan-2022/sample_submission.csv'
original_train_df = pd.read_csv(train_path)
original_test_df = pd.read_csv(test_path)
subm = pd.read_csv(sample_submission_path)
print(original_train_df.shape, original_test_df.shape)

In [ ]:
!pip install xlrd

In [ ]:
!pip install featurewiz --ignore-installed --no-deps

In [ ]:
import featurewiz as FW

In [ ]:
!pip install autoviz

In [ ]:
%%time
# The dates are read as strings and must be converted
for df in [original_train_df, original_test_df]:
    df['date'] = pd.to_datetime(df.date)
    df.set_index('date', inplace=True, drop=False)

In [ ]:
%%time
# Feature engineering
def engineer(df):
    """Return a new dataframe with the engineered features"""
    new_df = pd.DataFrame({'year': df.date.dt.year, # This feature makes it possible to fit an annual growth rate
                           'dayofyear': df.date.dt.dayofyear,
                           'wd4': df.date.dt.weekday == 4, # Friday
                           'wd56': df.date.dt.weekday >= 5, # Saturday and Sunday
                           'dec29': (df.date.dt.month == 12) & (df.date.dt.day == 29), # end-of-year peak
                           'dec30': (df.date.dt.month == 12) & (df.date.dt.day == 30),
                          })

    # Easter
    new_df['easter_week'] = False
    for year in range(2015, 2020):
        easter_date = easter.easter(year)
        easter_diff = df.date - np.datetime64(easter_date)
        new_df['easter_week'] = new_df['easter_week'] | (easter_diff > np.timedelta64(0, "D")) & (easter_diff < np.timedelta64(8, "D"))
    
    # Growth is country-specific
    #for country in ['Finland', 'Norway', 'Sweden']:
    #    new_df[f"{country}_year"] = (df.country == country) * df.date.dt.year
        
    # One-hot encoding (no need to encode the last categories)
    for country in ['Finland', 'Norway']:
        new_df[country] = df.country == country
    new_df['KaggleRama'] = df.store == 'KaggleRama'
    for product in ['Kaggle Mug', 'Kaggle Sticker']:
        new_df[product] = df['product'] == product
        
    # Seasonal variations (Fourier series)
    # The three products have different seasonal patterns
    dayofyear = df.date.dt.dayofyear
    for k in range(1, 100): # 100
        new_df[f'sin{k}'] = np.sin(dayofyear / 365 * 2 * math.pi * k)
        new_df[f'cos{k}'] = np.cos(dayofyear / 365 * 2 * math.pi * k)
        new_df[f'mug_sin{k}'] = new_df[f'sin{k}'] * new_df['Kaggle Mug']
        new_df[f'mug_cos{k}'] = new_df[f'cos{k}'] * new_df['Kaggle Mug']
        new_df[f'sticker_sin{k}'] = new_df[f'sin{k}'] * new_df['Kaggle Sticker']
        new_df[f'sticker_cos{k}'] = new_df[f'cos{k}'] * new_df['Kaggle Sticker']

    return new_df

train_df = engineer(original_train_df)
train_df['date'] = original_train_df.date
train_df['num_sold'] = original_train_df.num_sold.astype(np.float32)
test_df = engineer(original_test_df)
test_df.year = 2018 # no growth patch, see https://www.kaggle.com/c/tabular-playground-series-jan-2022/discussion/298318
#### George: You forgot to add two extra lines below for test_df. Without these two vars, train_df and test_df will be different.
test_df['date'] = original_test_df.date

features = test_df.columns
print(len(features))

In [ ]:
print(train_df.shape, test_df.shape)

In [ ]:
target = 'num_sold'

In [ ]:
train_df = train_df.reset_index(drop=True)
train_df.drop('date', axis=1, inplace=True)
train_df.head()

In [ ]:
test_df = test_df.reset_index(drop=True)
test_df.drop('date', axis=1, inplace=True)
test_df.head()

# Let's use Autoviz to select the best 100 features out of 607 features

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class

In [ ]:
AV = AutoViz_Class()
filename = ""
sep = ","
dft = AV.AutoViz(
    filename,
    sep=",",
    depVar=target,
    dfte=train_df,
    header=0,
    verbose=0,
    lowess=False,
    chart_format="svg",
    max_rows_analyzed=150000,
    max_cols_analyzed=100,
    save_plot_dir=None
)

In [ ]:
preds = dft.columns.tolist()
len(preds)

In [ ]:
train_best = train_df[preds]
test_best = test_df[preds[:-1]]
print(train_best.shape, test_best.shape)

# It look ~10 mins to select features in this dataset. We have 86 important features now

# This simple LightGBM model works wonders since it is highly effective in many competitions

In [ ]:
outputs = FW.simple_lightgbm_model(X_XGB=train_best[preds[:-1]], Y_XGB=train_best[target],
                               X_XGB_test=test_best[preds[:-1]], modeltype='Regression', verbose=-1)

# It took less than 1 min to build a model with RMSE average = 207 over 5 folds

In [ ]:
y_preds = outputs[0]
y_preds

In [ ]:
subm[target] = y_preds
subm.head()

In [ ]:
pd.DataFrame(y_preds).hist()

In [ ]:
train_df[target].hist()

In [ ]:
subm.to_csv('submission.csv',index=False)